#### This is to implement API for the XG Boost

In [2]:
import pandas as pd
import xgboost as xgb
from flask import Flask, request, jsonify
import json
from typing import Dict, Any
import logging

# Initialize Flask app
app = Flask(__name__)

class FraudPredictor:
    def __init__(self, model_path: str):
        self.model = xgb.XGBClassifier()
        self.model.load_model(model_path)
        self.required_features = self.model.feature_names_in_.tolist()
        self.threshold = 0.3  # Default threshold

    def predict_fraud(self, transaction_data: Dict[str, Any]) -> Dict[str, Any]:
        try:
            # Convert string booleans to integers
            converted_data = {}
            for key, value in transaction_data.items():
                if isinstance(value, str):
                    if value.upper() == "TRUE":
                        converted_data[key] = 1
                    elif value.upper() == "FALSE":
                        converted_data[key] = 0
                    else:
                        converted_data[key] = float(value)
                else:
                    converted_data[key] = value
            
            # Validate input
            missing_features = set(self.required_features) - set(converted_data.keys())
            if missing_features:
                raise ValueError(f"Missing features: {missing_features}")
            
            input_df = pd.DataFrame([converted_data])[self.required_features]
            
            # Predict
            prob = self.model.predict_proba(input_df)[0][1]
            
            return {
                "fraud_probability": float(prob),
                "is_fraud": bool(prob > self.threshold),
                "threshold": self.threshold,
                "status": "success"
            }
        except Exception as e:
            return {
                "status": "error",
                "message": str(e)
            }

# Initialize predictor
MODEL_PATH = "fraud_model.xgb"  # Update path if needed
predictor = FraudPredictor(MODEL_PATH)

@app.route('/predict', methods=['POST'])
def predict():
    """API endpoint for fraud prediction"""
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No input data provided"}), 400
        
        result = predictor.predict_fraud(data)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/update_threshold', methods=['POST'])
def update_threshold():
    """Endpoint to update decision threshold"""
    try:
        data = request.get_json()
        predictor.threshold = float(data['threshold'])
        return jsonify({"message": f"Threshold updated to {predictor.threshold}"})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

ModuleNotFoundError: No module named 'flask'